In [1]:
import os
from glob import glob
from collections import Counter

import pandas as pd
import numpy as np

In [2]:
survey = 'ZTF'

Read the data

In [3]:
path = '/home/iebecker/Desktop/iebecker_stuff/Work/ProcessZTF/notebooks/data/metadata_ztf.csv'
df = pd.read_csv(path).set_index('ID').rename({'teff':'T_eff', 'radius':'Radius'}, axis=1)
df.head()

,T_eff,Radius,N_1,N_2,Path,Class,N
ID,,,,,,,
ZTF17aaaaasi,-1.0,-1.0,386.0,291.0,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,LPV,677.0
ZTF17aaaaasx,-1.0,-1.0,132.0,52.0,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,LPV,184.0
ZTF17aaaaatf,-1.0,-1.0,219.0,156.0,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,LPV,375.0
ZTF17aaaadmx,-1.0,-1.0,488.0,356.0,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,LPV,844.0
ZTF17aaaaeal,-1.0,-1.0,334.0,204.0,/home/iebecker/Desktop/iebecker_stuff/Disks/Da...,LPV,538.0


Read the classification folds

In [4]:
folds = glob('../../../../CV_c/01_CreateFolds/{}/Folds/*'.format(survey))
folds.sort()
test_path = folds[-1]
folds = folds[:-1]
folds

['../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_1',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_2',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_3',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_4',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_5',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_6',
 '../../../../CV_c/01_CreateFolds/ZTF/Folds/Fold_7']

In [5]:
path_folds = './Folds'
if not os.path.exists(path_folds):
    os.mkdir(path_folds)

new_root = '/home/iebecker/Desktop/iebecker_stuff/Disks/'
test = pd.read_csv(test_path,usecols=['ID']).set_index('ID')
df_test = pd.merge(test, df,left_index=True, right_index=True).reset_index()
# df_test.Path = df_test.Path.str.replace("/home/",new_root ) # Fix path to work here

path_test = os.path.join(path_folds, 'test.csv')
df_test.to_csv(path_test, index=False, index_label=False)

for n,fold in enumerate(folds):
    # Read classifier folds
    train = pd.read_csv(os.path.join(fold, 'train.csv'), usecols=['ID']).set_index('ID')
    val = pd.read_csv(os.path.join(fold, 'val.csv'), usecols=['ID']).set_index('ID')

    # Join with new physical params
    df_train = pd.merge(train, df,left_index=True, right_index=True).reset_index()
    df_val = pd.merge(val, df,left_index=True, right_index=True).reset_index()
    # df_train.Path = df_train.Path.str.replace("/home/",new_root )
    # df_val.Path = df_val.Path.str.replace("/home/",new_root )

    
    path_folds_ = os.path.join(path_folds, 'Fold_'+str(n+1))
    if not os.path.exists(path_folds_):
        os.mkdir(path_folds_)
        
    path_train = os.path.join(path_folds_, 'train.csv')
    path_val = os.path.join(path_folds_, 'val.csv')
    
    df_train.to_csv(path_train, index=False, index_label=False)
    df_val.to_csv(path_val, index=False, index_label=False)